In [ ]:
#| default_exp Utils.network_architecture

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Network Architecture

>  We define a custom Neural Network architecture to train our RL agent.

In [ ]:
#| export

import torch
import torch.nn as nn
from d3rlpy.models.encoders import EncoderFactory
from typing import Tuple

We define here 2 constants to represent the shape of the new numerical observations and the number of units in each hidden layer in our custom Deep Neural Network.

In [ ]:
#| export
NEW_OBSERVATION_SHAPE = (18,)
N_UNITS = 64

## CustomEncoder Class

We are constructing a custom `Deep Neural Network` to encode numerical observations in order to train our `RL Agent` (processed by the `Scaler` defined in the Scaler Notebook).

In this class, we intend to overwrite the observation shape because we initially represented an observation shape of `(30,)` in our environment, and since it is preprocessed and transformed by the `Scaler` to gather `Teams features`, `1X2` and `Asian Handicap` odds, the shape has changed, and we need to overwrite it with the new numerical observation shape. The reason for this is that, during training, when the `D3rlpy` internal packages create the `EncoderFactory`, the `Neural Network Architecture` is automatically determined depending on the shape of the `GYM environment` observation.

In [ ]:
#| export


class CustomEncoder(nn.Module):
    "Set up a custom Neural Network to train RL agent."

    def __init__(
        self,
        observation_shape: Tuple,  # Environment observation shape, shape=(30,).
        feature_size: int,  # Number of network outputs (number of environment actions).
        n_units: int = N_UNITS,  # Number of units in each hidden layer.
    ) -> None:
        "Init Neural Network architecture."
        # Initialize self._modules as OrderedDict.
        super(CustomEncoder, self).__init__()
        # Overwrite the observation shape.(observation of the env = (30,))
        # The numerical observation that we processed has a shape of NEW_OBSERVATION_SHAPE.
        observation_shape = NEW_OBSERVATION_SHAPE
        self.feature_size = feature_size
        # First Layer.
        self.fc1 = nn.Linear(observation_shape[0], 64)
        # 2nd Layer.
        self.fc2 = nn.Linear(64, feature_size)

    def forward(
        self,
        x: torch.Tensor,  # Numerical observation.
    ) -> torch.Tensor:
        "Process inputs"
        # Apply ReLU in each layer.
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        return h

    # THIS IS IMPORTANT!(for EncoderFactory).
    def get_feature_size(self) -> int:
        "Returns the number of network outputs."
        return self.feature_size

In [ ]:
#| export


class CustomEncoderFactory(EncoderFactory):
    "D3rlpy Custom Encoder."
    # This is necessary to override the EncoderFactory.
    TYPE = "custom"

    def __init__(
        self,
        feature_size: int,  # Number of network outputs (number of environment actions).
    ):
        "Init Encoder"
        # Number of Network outputs.
        self.feature_size = feature_size

    def create(
        self,
        observation_shape: Tuple,  # Environment observation, shape=(30,).
    ):
        # Return the custom DNN.
        return CustomEncoder(observation_shape, self.feature_size)

    def get_params(
        self,
        deep: bool = False,  # Flag to deeply copy objects.
    ):
        "Serialize neural network configuration."
        return {"feature_size": self.feature_size}

In [ ]:
#| hide
from nbdev import nbdev_export

nbdev_export()